In [2]:
!pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install sentence-transformers faiss-cpu numpy

   ---------------------------------------- 1.8/1.8 MB 2.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
Looking in indexes: https://download.pytorch.org/whl/cpu


In [4]:
import json
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import os

# Load your chunked data
with open("chunked.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

# Load multilingual embedding model
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Generate embeddings
print("Generating embeddings...")
embeddings = model.encode(chunks, show_progress_bar=True)

# Convert to FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)  # L2 = Euclidean distance
index.add(np.array(embeddings))

# Save FAISS index
faiss.write_index(index, "faiss_index.index")

# Save metadata (to map index to chunks later)
with open("index_metadata.json", "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)

print("✅ Vector store created and saved successfully!")


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Generating embeddings...


Batches: 100%|██████████| 7/7 [00:27<00:00,  3.87s/it]


✅ Vector store created and saved successfully!


In [5]:
pip install openai


   -------------------------------------- 764.4/764.4 kB 536.0 kB/s eta 0:00:00
   ---------------------------------------- 2.0/2.0 MB 451.4 kB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0



In [10]:
!pip install transformers
!pip install torch

In [12]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Load FAISS and metadata
with open("index_metadata.json", "r", encoding="utf-8") as f:
    chunk_texts = json.load(f)

index = faiss.read_index("faiss_index.index")
model_embed = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Load local model: FLAN-T5
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model_gen = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

# RAG function using local model
def retrieve_and_answer_local(query, k=3):
    # Embed the query
    query_vec = model_embed.encode([query])
    D, I = index.search(np.array(query_vec), k)
    retrieved_chunks = [chunk_texts[i] for i in I[0]]
    context = "\n\n".join(retrieved_chunks)

    # Prompt for the local model (FLAN-T5 uses instruction format)
    prompt = f"""প্রশ্নের উত্তর দিন প্রাসঙ্গিক তথ্য ব্যবহার করে।

প্রাসঙ্গিক তথ্য:
{context}

প্রশ্ন: {query}

উত্তর:"""

    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    outputs = model_gen.generate(**inputs, max_new_tokens=100)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.strip()

# CLI interface
while True:
    user_query = input("\n❓ আপনার প্রশ্ন লিখুন (বাংলা বা English): ")
    if user_query.lower() in ["exit", "quit", "q"]:
        break
    answer = retrieve_and_answer_local(user_query)
    print(f"✅ উত্তর: {answer}")


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


✅ উত্তর: 
✅ উত্তর: 
✅ উত্তর: 


KeyboardInterrupt: Interrupted by user

In [13]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load chunk data
with open("index_metadata.json", "r", encoding="utf-8") as f:
    chunk_texts = json.load(f)

# Load FAISS index
index = faiss.read_index("faiss_index.index")

# Load multilingual embedding model
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Load FLAN-T5 model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
generator_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

# Final retrieve + generate function
def retrieve_and_answer_local(query, k=3):
    query_vec = embedding_model.encode([query])
    D, I = index.search(np.array(query_vec), k)
    retrieved_chunks = [chunk_texts[i] for i in I[0]]
    context = "\n".join(retrieved_chunks)

    prompt = f"""Answer the question in Bangla based on the following context:\n\n{context}\n\nQuestion: {query}\nAnswer:"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    outputs = generator_model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=100,
        do_sample=False,
        early_stopping=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

# Interface
while True:
    user_query = input("\n❓ প্রশ্ন লিখুন: ")
    if user_query.lower() in ["exit", "quit", "q"]:
        break
    answer = retrieve_and_answer_local(user_query)
    print(f"✅ উত্তর: {answer}")


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ উত্তর: 


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ উত্তর: 


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ উত্তর: 


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ উত্তর: 
